In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Importing Data

Loading the dataset

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Dataset can be retreived from https://bit.ly/UM-bmid

In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/BMI Research/gentwo_simple_clean_metadata.csv")
dataset.head(5)

## Input Measurements (S-Parameters)

We now load and pre-process the inputs to the network, which are the S-parameters \\
This will require turning the values from the frequency domain to the time domain, in order to reduce their size. \\
The code for this transformation can be found here: \\
https://github.com/UManitoba-BMS/UM-BMID/blob/5f58d866d205dd957c21d85b86002e9e83386806/umbmid/sigproc.py \\

https://github.com/UManitoba-BMS/UM-BMID/blob/5f58d866d205dd957c21d85b86002e9e83386806/umbmid/iczt.m \\

Example usage found here: \\
https://github.com/UManitoba-BMS/UM-BMID/blob/5f58d866d205dd957c21d85b86002e9e83386806/umbmid/build.py

Here, we load the processed s11 and s21 data 

In [ ]:
loaded_arr = np.loadtxt("/content/drive/MyDrive/BMI Research/s11.txt") 
s11_final = loaded_arr.reshape( 
    loaded_arr.shape[0], loaded_arr.shape[1] // 72,72)
loaded_arr = np.loadtxt("/content/drive/MyDrive/BMI Research/s21.txt") 
s21_final = loaded_arr.reshape( 
    loaded_arr.shape[0], loaded_arr.shape[1] // 72,72)

##Preparing Dataframe:

Removing irrelevant features

In [ ]:
dataset.drop('date', axis=1, inplace= True)
dataset.drop('n_expt', axis=1, inplace= True)
dataset.drop('id', axis=1, inplace= True)
dataset.drop('phant_id', axis=1, inplace= True)
dataset.drop('tum_shape', axis=1, inplace= True)
dataset.drop('adi_ref_id', axis=1, inplace= True)
dataset.drop('emp_ref_id', axis=1, inplace= True)
dataset.drop('n_session', axis=1, inplace= True)
dataset.drop('ant_z', axis=1, inplace= True)
dataset.drop('fib_ref_id', axis=1, inplace= True)
dataset.drop('tum_in_fib', axis=1, inplace= True)
dataset.drop('tum_rad', axis=1, inplace= True)
dataset.drop('tum_y', axis=1, inplace= True)
dataset.drop('tum_z', axis=1, inplace= True)
dataset.drop(dataset.filter(regex="Unnamed"),axis=1, inplace=True)
dataset.head()

Replacing the NaN values by "0"

In [ ]:
dataset['fib_ang'] = dataset['fib_ang'].fillna(0)
dataset['adi_x'] = dataset['adi_x'].fillna(0)
dataset['adi_y'] = dataset['adi_y'].fillna(0)
dataset['fib_x'] = dataset['fib_x'].fillna(0)
dataset['fib_y'] = dataset['fib_y'].fillna(0)
dataset['birads'] = dataset['birads'].fillna(2)
dataset.tail(5)

In [ ]:
dataset = dataset.dropna(subset=['tum_x'])
dataset

In [ ]:
X = s11_final
Y = s21_final
X_1 = []
X_2 = []
for i in range(len(X)):
  if(i in dataset.index):
    X_1.append(X[i])
for i in range(len(Y)):
  if(i in dataset.index):
    X_2.append(Y[i])
X_1 = np.array(X_1)
X_2 = np.array(X_2)

In [ ]:
dataset = dataset.reset_index(drop=True)
dataset

In [ ]:
dataset.isnull().sum()

##Dataset Splitting

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing
import scipy

X_1 = scipy.stats.zscore(X_1)
X_2 = scipy.stats.zscore(X_2)
X_md = scipy.stats.zscore(dataset.iloc[:,3:].values)
Y = scipy.stats.zscore(dataset.iloc[:,0:3].values)

#FOR CONV LAYER
X_1 = X_1.reshape((X_1.shape[0], X_1.shape[1], X_1.shape[2], 1))
X_2 = X_1.reshape((X_2.shape[0], X_2.shape[1], X_2.shape[2], 1))

x_1_train, x_1_test, y_train, y_test = train_test_split(X_1,Y, test_size=0.1, random_state=42)
x_2_train, x_2_test, y_train, y_test = train_test_split(X_2,Y, test_size=0.1, random_state=42)
x_md_train, x_md_test, y_train, y_test = train_test_split(X_md,Y, test_size=0.1, random_state=42)

In [ ]:
print(x_1_train.shape)
print(x_2_train.shape)
print(x_md_train.shape)
print(x_1_test.shape)
print(x_2_test.shape)
print(x_md_test.shape)
print(y_train.shape)
print(y_test.shape)

#Model Training

###Importing libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import concatenate

###Model architecture

In [ ]:
#S11 Input
input_1 = keras.Input(x_1_train[0].shape)
#S21 Input
input_2 = keras.Input(x_2_train[0].shape)
#metadata
input_3 = keras.Input(x_md_train[0].shape)

dense3 = Dense(14, activation = 'relu')(input_3)
dense4= Dense(10, activation = 'relu')(dense3)
dense5= Dense(4, activation = 'relu')(dense4)

#First branch
conv1_1 = Conv2D(1, (5,5), activation = 'relu')(input_1)
pool1_1 = MaxPooling2D()(conv1_1)
flatten_1_1 = Flatten()(pool1_1)

#Second branch
conv2_1 = Conv2D(1, (5,5), activation = 'relu')(input_2)
pool2_1 = MaxPooling2D()(conv2_1)
flatten_2_1 = Flatten()(pool2_1)

#Combine branches
combined = concatenate([flatten_1_1, flatten_2_1,dense5])

#Dense layers
d1 = Dense(300, activation="relu")(combined)
d2 = Dense(200, activation="relu")(d1)
d3 = Dense(100, activation="relu")(d2)
d7 = Dropout(0.25)(d3)
d4 = Dense(50, activation="relu")(d7)
d6 = Dropout(0.25)(d4)
d5 = Dense(20, activation="relu")(d6)
output = Dense(3, activation="linear")(d5)

#Overall model
model = keras.Model(inputs= [input_1, input_2, input_3], outputs=output, name="tumor localization net")

In [ ]:
keras.utils.plot_model(model, show_shapes=True)

###Model compilation and training

In [ ]:
model.compile(loss="mse", optimizer='adam', metrics = ['mse'])

In [ ]:
history = model.fit(x=[x_1_train, x_2_train, x_md_train], y=y_train, epochs=150, batch_size=64, validation_split=0.1)

In [ ]:
import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(history.history['mse'], 'b', linewidth=2)
plt.plot(history.history['val_mse'], 'r',  linewidth=2)
plt.ylabel('MSE', fontsize=15)
plt.xlabel('Epoch', fontsize=15)
plt.legend(['MSE', 'Validation MSE'])
plt.grid(linestyle='-', linewidth=0.5)
plt.savefig('tumor_localization.eps', dpi=300, format='eps')

###Model evaluation

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
preds = model.predict(x = [x_1_test, x_2_test, x_md_test])
print("R2 score : %.2f" % r2_score(y_test,preds))
print("Mean squared error: %.2f" % mean_squared_error(y_test,preds))